In [12]:
#!pip install -q -U google-genai

In [13]:
#!pip install google-generativeai

In [1]:
#!pip install pandas

In [20]:
import json 
import os
import google.generativeai as genai
import time
import pandas as pd

#### Using Google Gemini to generate a sintetic dataset for expense classification

In [11]:
categories_dict = {
  "Housing & Utilities": [
    "Rent/Mortgage",
    "Property Taxes",
    #"Home Insurance",
    "Home Maintenance & Repairs",
    #"Home Security Systems",
    "Electricity",
    "Water & Sewer",
    #"Natural Gas",
    #"Trash Collection",
    "Internet, phone and streaming services",
    #"Cable/Streaming Services",
    #"Phone (Mobile & Landline)"
  ],
  "Food & Dining": [
    "Groceries",
    "Restaurants, Take away and Dining Out",
    #"Fast Food",
    #"Coffee Shops",
    #"Bars & Nightlife",
    #"Meal Delivery (DoorDash, Uber Eats, etc.)",
    #"Work/School Cafeteria"
  ],
  "Transportation": [
    "Fuel/Gasoline",
    "Public Transport (Bus, Train, Subway)",
    "Ride-Sharing (Uber, Lyft)",
    "Car Loan Payments",
    "Car Insurance",
    "Vehicle Maintenance & Repairs",
  ]
}

In [28]:
lines = []

In [39]:
key = 'YOUR KEY GOES HERE'
for macro_category in categories_dict.keys():
    micro_categories = categories_dict[macro_category]
    for micro_category in micro_categories:
        prompt = f"""The goal is to create a sintetic dataset for expenditure classification.
        
        The dataset will have a simulated human prompt describing an expense and its classification.
        
        For example: 
        human string: unleaded gasoline shell 
        category: Fuel
        
        Use this JSON schema:
        
        category = 'category': str, 'human_strings': list[str]
        Return: list[]
        
        Create 100 hundred dataset entries for the category: {micro_category}.
        
        Include variations in writting that busy humans would do
        
        """
        
        
        generation_config = {
          "temperature": 1,
          "top_p": 0.95,
          "top_k": 40,
          "max_output_tokens": 8192,
          "response_mime_type": "application/json",
        }
        
        
        client = genai.Client(api_key=key)
        response = client.models.generate_content(
            model='gemini-1.5-flash',
            contents=prompt,
            config=generation_config
        )
        
        # Use the response as a JSON string.
        #print(response.text)
        dataset = json.loads(response.text)
        print(dataset[0:5])
        for entry in dataset:
            cat = entry['category']
            for human_string in entry['human_strings']:
                lines.append([cat, macro_category, human_string])
        time.sleep(10)

[{'category': 'Public Transport', 'human_strings': ['bus ticket', 'one way bus ticket', 'round trip bus ticket', 'bus fare', 'bus pass', 'monthly bus pass', 'bus ticket to the city center', 'bus ride', 'bus journey', 'journey by bus']}, {'category': 'Public Transport', 'human_strings': ['train ticket', 'train fare', 'train pass', 'railway ticket', 'commuter train', 'express train ticket', 'train to the airport', 'train ride', 'train journey', 'journey by train']}, {'category': 'Public Transport', 'human_strings': ['subway ticket', 'metro ticket', 'underground ticket', 'subway fare', 'metro fare', 'tube ticket', 'subway pass', 'daily subway pass', 'subway ride', 'subway journey']}, {'category': 'Public Transport', 'human_strings': ['bus to work', 'train to work', 'subway to work', 'bus to home', 'train to home', 'subway to home', 'bus commute', 'train commute', 'subway commute', 'daily commute by bus']}, {'category': 'Public Transport', 'human_strings': ['bus ticket purchase', 'train ti

In [40]:
text_dataset = pd.DataFrame(data=lines, columns = ['micro_category', 'macro_category', 'human_strings'])

In [41]:
text_dataset

,micro_category,macro_category,human_strings
0,Rent/Mortgage,Housing & Utilities,rent august
1,Rent/Mortgage,Housing & Utilities,rent for august
2,Rent/Mortgage,Housing & Utilities,august rent payment
3,Rent/Mortgage,Housing & Utilities,rent
4,Rent/Mortgage,Housing & Utilities,rent payment
...,...,...,...
3429,Vehicle Maintenance & Repairs,Transportation,automatic emergency braking installation
3430,Vehicle Maintenance & Repairs,Transportation,installed a collision avoidance system
3431,Vehicle Maintenance & Repairs,Transportation,collision avoidance system installation
3432,Vehicle Maintenance & Repairs,Transportation,installed a driver assistance system


In [42]:
text_dataset['micro_category'].unique()

array(['Rent/Mortgage', 'Property Taxes', 'Home Maintenance & Repairs',
       'Electricity', 'Water & Sewer',
       'Internet, phone and streaming services', 'Groceries',
       'Restaurants, Take away and Dining Out', 'Fuel/Gasoline',
       'Public Transport', 'Ride-Sharing', 'Car Loan Payments',
       'Car Insurance', 'Vehicle Maintenance & Repairs'], dtype=object)

In [21]:
text_dataset = pd.DataFrame(columns = ['micro_category', 'macro_category', 'human_strings'])

In [43]:
text_dataset.to_csv('expense_classification.csv', sep = '|')